<a href="https://colab.research.google.com/github/Nawrin2k16/Dungeon_Quest/blob/main/Dungeon_Quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code implements a Dungeon-Quest game where the player navigates through procedurally generated grids, avoiding monsters and obstacles, collecting treasure, and progressing through levels with increasing difficulty. I will now explain my code implementation, step by step, in detail.


In [165]:
!pip install keyboard numpy

If you already have keyboard numpy installed, please comment this part of the code. I used it to install keyboard numpy in colab.

In [166]:
from types import EllipsisType
import numpy as np
import random
import time
import heapq

# movement directions
directions = {
    'u': (-1, 0),  # up
    'd': (1, 0),   # down
    'l': (0, -1),  # left
    'r': (0, 1)    # right
}

In this part of the code, I implemented a dictionary called `directions` to define the possible movements within the "Dungeon-Quest" game. Each key corresponds to a movement direction: 'u' for up, 'd' for down, 'l' for left, and 'r' for right. The values are tuples that represent the changes in the row and column indices of the game grid. This setup allows me to easily compute the next position for a player or monsters by adding the tuple values to their current position.

In [167]:
def check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves):
    #Check if the game has reached a win or game-over state."""
    print(f"Moves counter: {moves_counter} and Total Moves: {total_moves}")

    if moves_counter >= total_moves:
        print("Player ran out of moves!😢 You died!💀")
        return -1  # Game over with player running out of moves

    elif moves_counter < total_moves and player_pos == treasure_pos:
        print("✨✨Player reached the treasure!🎉 You won!🌟")
        return 1  # Game over with player win

    for monster_pos in monsters_pos:
        if monster_pos == player_pos:
            print("Monster caught the player!😞 You died!💀")
            return -1  # Game over with monster catching player

    for monster_pos in monsters_pos:
        if monster_pos == treasure_pos:
            print("Monster reached the treasure!😭 You died!💀")
            return -1  # Game over with monster reaching treasure

    return False  # Game continues

In this part of the code, I updated the `check_game_status` function to provide more engaging feedback and determine whether the game has reached a win or game-over state:

1. **Move Limit Check:**  
   If the player has used up all the allowed moves (`moves_counter >= total_moves`), the game ends with a loss, and a message is displayed: *"You died!"*.

2. **Treasure Reached:**  
   If the player reaches the treasure before running out of moves, the game ends with a win, and a celebratory message is shown: *"You won!"*.

3. **Monster Catch:**  
   If any monster's position matches the player's position, the game ends in a loss, with the message: *"You died!"*.

4. **Monster Reaches Treasure:**  
   If any monster reaches the treasure before the player, the game also ends with a loss: *"You died!"*.

5. **Game Continuation:**  
   If none of these conditions are met, the game continues, and no final message is printed.

I added emojis to make the messages more dynamic and fun, increasing player engagement.

In [168]:
def reinforcement_learning(grid, start, goal, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    # Q-table initialization
    q_table = {}
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            q_table[(i, j)] = {direction: 0 for direction in directions}

    # Action selection policy
    def choose_action(state):
        if random.uniform(0, 1) < epsilon:
            return random.choice(list(directions.keys()))
        #print ("Choose Action: ", max(q_table[state], key=q_table[state].get))
        return max(q_table[state], key=q_table[state].get)

    # Training episodes
    for episode in range(episodes):
        state = start
        while state != goal:
            action = choose_action(state)
            dx, dy = directions[action]
            next_state = (state[0] + dx, state[1] + dy)

            # for valid move within grid bounds and not into obstacles
            if (0 <= next_state[0] < grid.shape[0] and 0 <= next_state[1] < grid.shape[1] and grid[next_state] != 1):
                reward = 10 if next_state == goal else -1  # Reward for reaching goal, penalty otherwise
                max_future_q = max(q_table[next_state].values())
                q_table[state][action] += alpha * (reward + gamma * max_future_q - q_table[state][action])
                state = next_state
            else:
                # a small penalty for invalid moves
                q_table[state][action] -= 1
        #print("Q table: ", q_table)

    # Function to determine the next position for the monster
    def get_next_position(monster_pos):
        best_action = max(q_table[monster_pos], key=q_table[monster_pos].get)
        dx, dy = directions[best_action]
        next_pos = (monster_pos[0] + dx, monster_pos[1] + dy)

        if 0 <= next_pos[0] < grid.shape[0] and 0 <= next_pos[1] < grid.shape[1] and grid[next_pos] != 1:
            #print("Next position in function get_next_position", next_pos)
            return next_pos
        print("No valid move")
        return monster_pos  # If no valid move, stay in current position

    #print("get_next_position(start): ", get_next_position(start))
    #print("q_table: ", q_table)

    return get_next_position(start)

In this part, I implemented a **Reinforcement Learning (RL) agent** using Q-learning to train monsters for smarter movement in the game. Here's how it works:

1. **Q-Table Initialization:** I created a Q-table, where each state (grid position) is a key, and its value is a dictionary of Q-values for each possible direction. Initially, all Q-values are set to zero.

2. **Action Selection Policy:** The agent uses an epsilon-greedy strategy for choosing actions:
   - With a probability of `epsilon`, it explores by choosing a random direction.
   - Otherwise, it exploits by choosing the direction with the highest Q-value.

3. **Training Loop:** Over multiple episodes, the agent learns:
   - It starts at the given starting position (`start`) and moves until it reaches the goal.
   - Rewards are assigned: +10 for reaching the goal, -1 for other valid moves, and a small penalty for invalid moves.
   - The Q-values are updated using the **Bellman equation**, balancing immediate rewards and future potential rewards (`gamma` as the discount factor).

4. **Handling Invalid Moves:** The agent avoids grid boundaries and obstacles (cells with `1`) and learns the best path to reach the goal while penalizing invalid moves.

5. **Learned Policy Application:** After training, the `get_next_position` function determines the next move for the monster based on the learned Q-table, ensuring the move is valid and optimal.

In [169]:
def reconstruct_path(came_from, current):
    """Reconstruct the path from start to goal."""
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    total_path.reverse()
    return total_path

def heuristic(a, b):
    """Calculate the Manhattan distance heuristic for A*."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

I implemented helper functions to support pathfinding with the A* algorithm:

1. **`reconstruct_path`:**  
   This function reconstructs the path from the goal to the start by backtracking through the `came_from` dictionary, which stores each node's predecessor. Starting from the `current` (goal), it iteratively adds each predecessor to the `total_path` until the start node is reached. The path is then reversed to present it from start to goal.  

2. **`heuristic`:**  
   This function calculates the **Manhattan distance** heuristic between two points `a` and `b` on a grid. It is defined as the sum of the absolute differences in their row and column coordinates. This heuristic is particularly suited for grid-based environments, providing a measure of the estimated cost to reach the goal from a given point.

These functions form the backbone of efficient pathfinding, helping the A* and bfs algorithm (occasionally) compute optimal paths from the start to the goal while minimizing computational overhead.

In [170]:
def a_star_pathfinding(grid, start, goal):
    """A* pathfinding algorithm to find the shortest path from start to goal."""
    open_set = []
    heapq.heappush(open_set, (0, start))  # (cost, position)
    came_from = {}

    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            return reconstruct_path(came_from, current)

        for direction in directions.values():
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            # Check if the neighbor is within bounds and not a wall
            if (0 <= neighbor[0] < grid.shape[0] and
                0 <= neighbor[1] < grid.shape[1] and
                grid[neighbor] != 1):

                tentative_g_score = g_score[current] + 1  # Cost is 1 for each step
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)

                    if neighbor not in [i[1] for i in open_set]:
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []  # Return empty path if there is no path to goal

The **A\* pathfinding algorithm** computes the shortest path from the `start` position to the `goal` in the grid. Here's how it works:

1. **Initialization:**  
   - I used a priority queue (`open_set`) to store nodes based on their total estimated cost (`f_score`).
   - `g_score` tracks the cost of the shortest known path from the start to each node.
   - `f_score` estimates the total cost from the start to the goal via each node, combining `g_score` and the heuristic.

2. **Exploration Loop:**  
   - The node with the lowest `f_score` is selected for exploration.
   - If the goal is reached, the path is reconstructed using the `came_from` dictionary and returned.

3. **Neighbor Evaluation:**  
   - For each valid neighboring cell (not out of bounds or a wall), the algorithm calculates a tentative `g_score` for the new path.
   - If this path is better than any previously known path to that neighbor, it updates the scores and records the current node as the predecessor (`came_from`).

4. **Priority Queue Management:**  
   - Neighbors not already in `open_set` are added to the queue with their updated `f_score`.

5. **Failure Case:**  
   - If no path to the goal exists, the function returns an empty path.

This implementation ensures efficient navigation by balancing exploration and exploitation using the heuristic and is well-suited for grid-based challenges like "Dungeon-Quest."

In [171]:
def bfs_pathfinding(grid, start, target):
    """Perform BFS to find a path from start to target on the grid."""
    queue = [start]
    visited = {start}
    came_from = {}

    while queue:
        current = queue.pop(0)
        if current == target:
            return reconstruct_path(came_from, target)

        x, y = current
        for dx, dy in directions.values():
            nx, ny = x + dx, y + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and (nx, ny) not in visited and grid[nx, ny] != 1:
                visited.add((nx, ny))
                came_from[(nx, ny)] = current
                queue.append((nx, ny))

    return []  # Return empty if no path found

def unoptimized_bfs_pathfinding(grid, start, target):
    queue = [[start]]
    visited = {start}

    while queue:
        path = queue.pop(0)
        current = path[-1]

        if current == target:
            return path

        # Explore neighbors in random order (I want it to sometimes lead to non-optimal paths)
        x, y = current
        neighbors = list(directions.values())
        random.shuffle(neighbors)  # Randomize order to make it unoptimized
        for dx, dy in neighbors:
            neighbor = (x + dx, y + dy)

            if (0 <= neighbor[0] < grid.shape[0] and
                0 <= neighbor[1] < grid.shape[1] and
                neighbor not in visited and grid[neighbor] != 1):

                visited.add(neighbor)
                queue.append(path + [neighbor])

    return []  # Return empty if no path found

In this section, I implemented two versions of Breadth-First Search (BFS) for pathfinding in the grid. Both approaches are designed to explore all possible paths, ensuring a solution is found if one exists, but they differ in optimization and behavior:

---

### 1. **`bfs_pathfinding` (Optimized BFS):**
- **Purpose:** Finds the shortest path from the `start` to the `target` using a standard BFS approach.
- **Implementation Highlights:**
  - Uses a queue to explore neighbors level by level, ensuring the shortest path is found first.
  - Tracks visited cells in a set to avoid revisiting and prevent infinite loops.
  - Stores predecessors in the `came_from` dictionary to reconstruct the path once the target is reached.
  - Only considers valid moves (within grid bounds and not blocked by walls).
- **Output:** Returns the shortest path or an empty list if no path exists.

---

### 2. **`unoptimized_bfs_pathfinding` (Unoptimized BFS):**
- **Purpose:** Simulates an unoptimized BFS by exploring neighbors in a random order, potentially leading to longer paths.
- **Implementation Highlights:**
  - The queue stores paths instead of individual nodes. Each path is extended as neighbors are added.
  - Neighbors are randomized before exploration to simulate non-optimal behavior.
  - Does not explicitly use a `came_from` structure; paths are directly built during traversal.
- **Output:** Returns the first path found to the target, which may not be the shortest, or an empty list if no path exists.

---

### Use Cases:
- **`bfs_pathfinding`:** Used for monster's movements in lower level.
- **`unoptimized_bfs_pathfinding`:** Used to find random position for obstacles while leaving a possible route between player and treasure, and between monster and player. Also used before placing treasure after certain interval to make sure an availabe route.

In [194]:
def initialize_level_elements(level):
    global player_character

    obstacles_list = ["🔥", "🪨", "🧱", "🌊", "🛑", "⬛"]
    monsters_list = ["👹", "😈", "👺", "🐉", "🧟", "👾"]
    empty_space_list = ["🟦", "🟩", "⬜", "🔲", "⚪"]
    treasures_list = ["💎", "🏆", "🗝️", "🪙", "👑", "💰"]

    # Messages for obstacles, monsters, and treasures
    obstacle_messages = {
        "🔥": "through the scorching inferno of lava (🔥)",
        "🪨": "as you dash across the rugged field of stones (🪨)",
        "🧱": "in the shadow of the crumbling brick wall (🧱)",
        "🌊": "navigating the treacherous waters of the flooded river (🌊)",
        "🛑": "avoiding the deadly traps in the perilous field (🛑)",
        "⬛": "in the depths of the ominous dark void (⬛)"
        }

    monster_messages = {
        "🐉": "venemous snake (🐉)",
        "🧟": "vicious zombie (🧟)",
        "😈": "evil demon (😈)",
        "👾": "space invader (👾)",
        "👺": "dangerous oni (👺)",
        "👹": "ugly ogre (👹)"
    }

    treasure_messages = {
        "💎": "precious diamond (💎)",
        "🏆": "sparkling trophy of gold (🏆)",
        "🗝️": "sole key to the treasure chest(🗝️)",
        "🪙": "glimmering golden coin (🪙)",
        "👑": "royal crown (👑)",
        "💰": "treasure bag filled with riches (💰)"
    }

    stone = random.choice(obstacles_list)
    monster = random.choice(monsters_list)
    players_list = ["🧑", "👩", "👨", "🧓", "👴"]

    if level == 1:
        print("Choose your character:")
        for i, p in enumerate(players_list, 1):
            print(f"{i}: {p}")
        player_choice = int(input("Enter a number (1-5): "))
        while player_choice not in range(1, 6):
            player_choice = int(input("Invalid choice! Please choose a number between 1 and 5: "))

        player_character = players_list[player_choice - 1]  # Stored the chosen player for future levels

    player = player_character

    empty_space = random.choice(empty_space_list)
    treasure = random.choice(treasures_list)

    # a special message based on selected monster, obstacle, and treasure
    print(f"Player, this time you have to save yourself from {level} {monster_messages[monster]}, "
          f"{obstacle_messages[stone]} and take the {treasure_messages[treasure]}, "
          f"before the {monster_messages[monster]} gets to it!")

    symbols = {0: empty_space, 1: stone, 2: player, 3: treasure, 4: monster}

    return symbols, stone, monster, player, treasure, empty_space


Here I tried implementing a function that sets up the environment for each level by randomly selecting obstacles, monsters, treasures, and a player character. Here's a breakdown of what each section does:

1. **Element Initialization:**
   - I defined lists for obstacles (`obstacles_list`), monsters (`monsters_list`), empty spaces (`empty_space_list`), and treasures (`treasures_list`), each containing various icons (emoji characters).
   
2. **Messages for Context:**
   - For each element (obstacles, monsters, and treasures), I created corresponding dictionaries (`obstacle_messages`, `monster_messages`, `treasure_messages`) to provide descriptive messages about the challenges and rewards the player will face.

3. **Player Character Selection (Level 1):**
   - If the player is starting level 1, they are prompted to choose their character from a list (`players_list`) containing different emoji characters.
   - I used an input loop to ensure the player selects a valid option (between 1 and 5).

4. **Element Randomization:**
   - For each level, the function randomly selects one obstacle, monster, treasure, and empty space, ensuring each playthrough is unique and provides different challenges.

5. **Message Display:**
   - A descriptive message is printed, giving the player context on what to expect for this level, such as the monster they're up against, the obstacle they must avoid, and the treasure they need to collect.

6. **Grid Representation:**
   - I created a `symbols` dictionary to map different game elements to integers (e.g., `0` for empty space, `1` for obstacle, `2` for player, `3` for treasure, and `4` for monster) for use in the game grid.

7. **Return Values:**
   - The function returns all the necessary variables: `symbols`, `stone`, `monster`, `player`, `treasure`, and `empty_space`, which will be used to set up the level in the game.

In [173]:
def print_grid(grid, symbols, stone):
    """Print the dungeon grid using emojis based on the given symbols."""
    # Top border
    print(stone * (grid.shape[1] + 2))

    for row in grid:
        # Side border and row content
        print(stone + "".join(f"{symbols[cell]}" for cell in row) + stone)

    # Bottom border
    print(stone * (grid.shape[1] + 2))

In this part of the code, I implemented the **`print_grid`** function to visually represent the dungeon grid using emojis. This function is responsible for displaying the current state of the game in a grid format.
If the grid is a 5x5 grid with the following content, this function would output a grid somewhat like this:

🧱🧱🧱🧱🧱🧱🧱\
🧱🟦🟦🟦🟦🟦🧱\
🧱👨🧱🧟🟦🟦🧱\
🧱🟦💎🟦🧱🟦🧱\
🧱🧱🟦🧱🟦🧱🧱\
🧱🟦🟦🟦🟦🟦🧱\
🧱🧱🧱🧱🧱🧱🧱

In [174]:
def add_random_obstacles(grid, path):
    """Add random obstacles to the grid, avoiding the cells in the specified path."""
    num_walls = int(grid.size * 0.2)  # 20% of the grid cells are walls
    for _ in range(num_walls):
        x, y = random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1)
        if grid[x, y] == 0 and (x, y) not in path:
            grid[x, y] = 1

I implemented `add_random_obstacles` here to enhance the grid environment by introducing walls, which serve as obstacles in "Dungeon-Quest." Here's how it works:

1. **Obstacle Count:**  
   - The number of walls (`num_walls`) is calculated as 20% of the total grid cells (`grid.size`).

2. **Random Placement:**  
   - For each obstacle, a random position `(x, y)` is generated within the grid boundaries.
   - Obstacles (value `1`) are only placed in cells that:
     - Are currently empty (`grid[x, y] == 0`).
     - Are not part of the specified `path` (to ensure the path remains traversable).

3. **Avoiding Path Conflicts:**  
   - The `path` parameter ensures critical gameplay routes, such as the player's planned path to the goal, are not obstructed by randomly placed walls.

In [175]:
def shift_obstacles(grid, player_pos, treasure_pos, monsters_pos, size):
    original_grid = grid.copy()
    add_random_obstacles(grid, [player_pos, treasure_pos] + monsters_pos)
    # I need to ensure paths are still valid for player and all monsters
    if not (unoptimized_bfs_pathfinding(grid, player_pos, treasure_pos) and
            all(unoptimized_bfs_pathfinding(grid, monster_pos, player_pos) for monster_pos in monsters_pos)):
        print("No valid path found after shifting obstacles.")
        grid = original_grid
    return grid


`shift_obstacles` is made to dynamically update the grid by shifting obstacles while maintaining valid gameplay paths. Here's how it works:

1. **Backup the Original Grid:**  
   - A copy of the current grid (`original_grid`) is saved in case the obstacle shift invalidates necessary paths.

2. **Add New Obstacles:**  
   - The grid is cleared and obstacles are randomly re-added using the `add_random_obstacles` function.
   - Obstacles are placed while ensuring that critical positions (player, treasure, and monsters) remain unobstructed.

3. **Path Validation:**  
   - The function ensures that:
     - The player can still reach the treasure (`unoptimized_bfs_pathfinding(grid, player_pos, treasure_pos)`).
     - Each monster can still reach the player (`unoptimized_bfs_pathfinding(grid, monster_pos, player_pos)` for all monsters).
   - If any path becomes invalid, the grid is reverted to the original state, and a warning message is printed.

4. **Return Updated Grid:**  
   - The function returns the modified grid if paths are valid; otherwise, it reverts to the original grid.

This approach adds dynamic complexity to the environment. And ensures that the game remains playable and fair.

In [176]:
def place_treasure_in_empty_cell(grid):
    empty_cells = [(i, j) for i in range(grid.shape[0]) for j in range(grid.shape[1]) if grid[i, j] == 0]
    if empty_cells:
        return random.choice(empty_cells)
    return None

This function simply finds a randomly chosen empty cell to place the treasure after iterating over the entire grid. If no empty cells are available (e.g., the grid is full), it returns `None`.

In [177]:
def place_treasure(grid, player_position):
    while True:
        treasure_position = place_treasure_in_empty_cell(grid)
        if treasure_position is None:
            return None  # No empty cell available, can't place treasure

        path = unoptimized_bfs_pathfinding(grid, player_position, treasure_position)
        if path:  # If a path is found
            grid[treasure_position] = 2  # Mark 2 for treasure
            return treasure_position
        else:
            continue # coming here means no path exists, so try again with a new random position for the treasure

The `place_treasure` function ensures that the treasure is placed in a reachable location on the grid. It randomly selects an empty cell and checks if there's a valid path from the player's position to the treasure using unoptimized BFS pathfinding. If a valid path is found, the treasure is placed on the grid, and its coordinates are returned. If no path exists, the function retries with a new random location. If no empty cells are available, it returns `None`, indicating the treasure cannot be placed.

In [178]:
def create_dungeon_grid(size, level):
    grid = np.zeros((size, size), dtype=int)

    player_pos = (0, 0)
    treasure_pos = (size - 1, size - 1)
    grid[player_pos] = 2  #  player position
    grid[treasure_pos] = 3  # treasure position

    # a path must be from player to treasure
    path1 = unoptimized_bfs_pathfinding(grid, player_pos, treasure_pos)

    # path on grid (for testing, could be removed in my final version)
    for cell in path1:
        if grid[cell] == 0:  # Leave player and treasure positions as-is
            grid[cell] = 0  # Keep path cells empty

    # Place monsters based on level (level defines how many monsters)
    monsters_pos = []
    for _ in range(level):  # Add one extra monster for each level up :hehe (evil laughter...muhahaahaa)
        empty_positions = [(i, j) for i in range(size) for j in range(size) if grid[i, j] == 0]
        monster_pos = random.choice(empty_positions)
        grid[monster_pos] = 4  #  monster position
        monsters_pos.append(monster_pos)

    for monster_pos in monsters_pos:
        path2 = unoptimized_bfs_pathfinding(grid, monster_pos, treasure_pos)
        for cell in path2:
            if grid[cell] == 0:  # Leave monster and treasure positions as-is
                grid[cell] = 0

    # Now add random obstacles without blocking the path
    add_random_obstacles(grid, set(path1 + path2))

    return grid, player_pos, treasure_pos, monsters_pos

The `create_dungeon_grid` function generates a dungeon grid that ensures the player can reach the treasure while considering difficulty levels. It starts by creating a grid and placing the player at the top-left and the treasure at the bottom-right. It uses unoptimized BFS to ensure a valid path between the player and treasure. Monsters are added based on the level, with their positions also checked to ensure they can reach the treasure. Random obstacles are placed without blocking crucial paths. The function returns the updated grid and the positions of the player, treasure, and monsters.

In [179]:
def move_player(grid, player_pos):
    last_valid_position = player_pos  # Store the last valid position
    while True:
        direction = input("Enter direction (u=up, d=down, l=left, r=right): ").strip().lower()
        if direction not in directions:
            print("Invalid input! Please use 'u', 'd', 'l', or 'r'.")
            continue

        new_pos = (player_pos[0] + directions[direction][0], player_pos[1] + directions[direction][1])

        # Check if the new position is valid (inside grid and not a blockage)
        if (0 <= new_pos[0] < grid.shape[0] and 0 <= new_pos[1] < grid.shape[1]):
            if (grid[new_pos] == 0 or grid[new_pos] == 3):
                grid[player_pos] = 0   # Mark old position as empty
                grid[new_pos] = 2      # Move player to new position
                last_valid_position = new_pos  # Update last valid position
                return new_pos

            elif (grid[new_pos] == 4):
                print("Player jumped on the monster and attempted suicide!☠️ You died!! 💀")
                return -1
        else:
            print("Invalid Move! Please try again.")
            print(f"Current Position: {last_valid_position}")  # Show last valid position

The `move_player` function manages the player's movement in the dungeon grid. It prompts the player for a direction and checks if the move is valid, considering grid boundaries, blocked cells, and monster encounters. If the move is valid, the player's position is updated on the grid. If the move is invalid (out of bounds or blocked), the player is prompted to try again, and the last valid position is shown. If the player rushes to a monster position, the game ends. The function ensures proper movement while enforcing game rules.

In [180]:
def move_monster(grid, monster_pos, player_pos, treasure_pos, level = 1):
    """Move the monster towards the nearest target (player or treasure)."""
    if (level == 1):
        path_to_player = unoptimized_bfs_pathfinding(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = unoptimized_bfs_pathfinding(grid, monster_pos, treasure_pos)
        print("APPLYING BFS")
    elif (level == 2):
        path_to_player = a_star_pathfinding(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = a_star_pathfinding(grid, monster_pos, treasure_pos)
        print("APPLYING A*")
    else:
        path_to_player = reinforcement_learning(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = reinforcement_learning(grid, monster_pos, treasure_pos)

    # Select the shortest valid path (player or treasure)
    if treasure_pos and len(path_to_player) > len(path_to_treasure):
        path = path_to_treasure
    elif path_to_player:
        path = path_to_player
    else:
        # If there is no valid path to player or treasure, move randomly
        empty_neighbors = []
        for dx, dy in directions.values():
            nx, ny = monster_pos[0] + dx, monster_pos[1] + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx, ny] == 0:
                empty_neighbors.append((nx, ny))

        if empty_neighbors:
            new_pos = random.choice(empty_neighbors)  # Move to any random valid neighbor
            grid[monster_pos] = 0
            grid[new_pos] = 4
            return new_pos, False  # Monster moved randomly

        # If no valid neighbors, keep the monster in place
        return monster_pos, False

    # If path exists and has more than one step, move monster
    if path and len(path) > 1:
        if level < 3:
            new_pos = path[1]  # Move one step along the path
        else:
            new_pos = path  # If level is 1, move directly to the next step in path

        print("Position in monster move: ", new_pos)

        # Ensure monster only moves to empty spaces or player/treasure positions
        if grid[new_pos] == 0 or grid[new_pos] == 2 or grid[new_pos] == 3:
            grid[monster_pos] = 0
            grid[new_pos] = 4
            return new_pos

        print("Position in monster move: ", new_pos)

    return monster_pos  # If no valid move, return current position

The `move_monster` function controls the monster's movement based on the difficulty level and chosen pathfinding algorithm:

1. **Pathfinding Algorithms:**
   - **Level 1:** BFS is used to find the shortest path to the player or treasure, without considering obstacles.
   - **Level 2:** A* pathfinding optimizes the route based on distance and heuristics.
   - **Level 3:** Reinforcement learning determines the monster's movements based on prior experience.

2. **Path Selection:** The monster prioritizes moving toward the treasure if it has a shorter path, else it moves toward the player. If no valid path exists, it moves randomly.

3. **Movement:** The monster follows the shorter path, moving step-by-step toward its goal, and only moves into empty cells, the player, or the treasure (if exists). If blocked, it stays put.

4. **Edge Case:** If surrounded by walls, the monster remains in place.

In [181]:
def move_all_monsters(grid, monsters_pos, player_pos, treasure_pos, level):
    #Move all monsters towards the nearest target (player or treasure).
    new_monster_positions = []
    for monster_pos in monsters_pos:
        new_pos = move_monster(grid, monster_pos, player_pos, treasure_pos, level)
        new_monster_positions.append(new_pos)
    return new_monster_positions

The `move_all_monsters` function moves all monsters on the grid toward their target (either the player or the treasure):

1. **Iterate Over Monsters:** The function loops through the list of monster positions (`monsters_pos`).
2. **Move Each Monster:** It calls the `move_monster` function for each monster, which determines the best move using the pathfinding strategy defined by the current level.
3. **Update Positions:** The new positions from `move_monster` are stored in a list (`new_monster_positions`).
4. **Return Updated Positions:** Once all monsters have moved, the function returns the list of updated positions.

### Key Features:
- Each monster moves according to its pathfinding algorithm (BFS, A*, or reinforcement learning).
- The function updates all monsters' positions in one go, coordinating their movements and ensuring the game state is updated efficiently. This enhances the gameplay dynamics by handling monster movement collectively.

In [192]:
def main():
    size = 0
    print("😇😄 Welcome to the dungeon game! 😄😇 \nUse 'u' for up, 'd' for down, 'l' for left, 'r' for right. 🤓")
    while True:
        size = input("Enter the size of dungeon between 3 and 100: 😃")
        if size.isdigit():  # Check if input is a positive number
            size = int(size)
            if 3 <= size <= 100:
                break  # Exit the loop if size is valid (no need to nag the user anymore...)
            else:
                print("Invalid input! Please enter a number between 3 and 100.😔")
        else:
            print("Invalid input! Please enter a numeric value.😕")

    level = 1

    while level >= 1:
        flag = True
        print(f"Starting Level {level}!")
        grid, player_pos, treasure_pos, monsters_pos = create_dungeon_grid(size, level)
        treasure_disappear_counter = random.randint(1, size // 2)
        treasure_reappear_counter = random.randint(1, size // 3)

        symbols, stone, monster, player, treasure, empty_space = initialize_level_elements(level)
        print_grid(grid, symbols, stone)
        moves_counter = 0
        while True:
            player_pos = move_player(grid, player_pos)
            if player_pos == -1:
                print("💥Game over!💥")
                flag = False
                break
            moves_counter += 1
            total_moves = size * size // level

            if (moves_counter % size == 0):  # I just felt like shifting the obstacles with the same delay as dungeon size. Might change later!
                print("Shifting obstacles... Prepare yourself, Player.")
                grid = shift_obstacles(grid, player_pos, treasure_pos, monsters_pos, size)
            print("Player moved:")
            print_grid(grid, symbols, stone)

            if check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves):
                print(f"Level {level} completed!🙂 Moving to Level {level+1}🤩")
                level += 1  # Increase level
                size *= 2  # Double the grid size for the next level
                break

            # Treasure disappear and reappear logic
            if treasure_pos:
                treasure_disappear_counter -= 1
                if treasure_disappear_counter <= 0:
                    grid[treasure_pos] = 0  # Remove treasure from grid
                    treasure_pos = None  # Treasure is now gone
                    treasure_reappear_counter = random.randint(1, size // 3)  # Reset reappear counter
                    print(f"Treasure Disappearance Counter has reached 0.\nThe treasure has disappeared!😓 \nPlayer, continue to save yourself from the monster! \nUntil the treasure reappears, Don't die! \nTreasure Reappearance Counter: {treasure_reappear_counter}")
                else:
                    print(f"Treasure Disappearance Counter: {treasure_disappear_counter}")
            else:
                treasure_reappear_counter -= 1
                if treasure_reappear_counter <= 0:
                    treasure_pos = place_treasure(grid, player_pos)
                    if treasure_pos:
                        grid[treasure_pos] = 3  # Place treasure in new empty cell
                    treasure_disappear_counter = random.randint(1, size // 2)  # Reset disappear counter
                    #print(f"Treasure Reappearance Counter has reached 0. \nPlayer, Congratulations on surviving with no hope! \nThe treasure has reappeared! \nTreasure Disappearance Counter: {treasure_disappear_counter}")
                    print_grid(grid, symbols, stone)
                else:
                    print(f"Treasure Reappearance Counter: {treasure_reappear_counter}")


            monsters_pos = move_all_monsters(grid, monsters_pos, player_pos, treasure_pos, level)
            print("Monsters moved:")
            print_grid(grid, symbols, stone)

            if check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves) == -1:
                flag = False
                break

            print(f"Remaining Moves: {total_moves - moves_counter} ")
            print(f"Dungeon Shift counter: {size - moves_counter % size}")

            time.sleep(0.5)  # Short delay for readability

        if not flag:
            print("Press any key to continue or press 'n' to exit")
            choice = input().lower()
            if choice == 'n':
                print("Thanks for playing! Goodbye! 😖")
                break
            else:
                level = 1
                while True:
                    size = input("Enter the size of dungeon between 3 and 100: ")
                    if size.isdigit():  # Check if input is a positive number
                        size = int(size)
                        if 3 <= size <= 100:
                            break  # Exit the loop if size is valid
                        else:
                            print("Invalid input! Please enter a number between 3 and 100. 😔")
                    else:
                        print("Invalid input! Please enter a numeric value.😕")


The **`main()`** function is the entry point for the dungeon game. It manages the game loop, handles user input, and controls the flow of the game.

### Key Components:

1. **Dungeon Size Input:**
   - The game asks the player to input a dungeon size between 3 and 100. It validates the input to ensure it falls within the correct range.
   - Once the size is set, the game continues, and the player proceeds to the next stage.

2. **Level Handling:**
   - The game starts at **Level 1** and progresses to higher levels as the player completes each level.
   - For each level, a new dungeon grid is created using the **`create_dungeon_grid()`** function. The size of the dungeon grid changes as the player progresses (doubling in size after each level).

3. **Treasure and Monster Handling:**
   - **Treasure Mechanics:**
     - The treasure has a disappearance counter, meaning it can disappear at random intervals. Once it disappears, the player needs to continue playing until it reappears.
     - If the treasure reappears, it is placed back in the grid in an empty spot.
   - **Monsters:**
     - Monsters are placed in the grid and move toward the player and/or treasure. The **`move_all_monsters()`** function controls their movement each round.

4. **Player Movement:**
   - The player moves using the keys: 'u' for up, 'd' for down, 'l' for left, and 'r' for right.
   - The **`move_player()`** function handles the player's movement on the grid.
   
5. **Grid Shifting:**
   - Every few moves, the grid shifts, meaning obstacles are moved around the grid. This adds a level of unpredictability and challenge to the game.
   
6. **Game Over or Victory Check:**
   - After every move, the **`check_game_status()`** function is called to check if the player has won or lost. If the player runs out of moves, gets caught by a monster, or reaches the treasure, the game ends.

7. **Level Completion:**
   - When a player completes a level by reaching the treasure or avoiding monsters, the game moves to the next level. The grid size doubles, and a new level starts.

8. **User Choices for Continuing or Exiting:**
   - If the player dies, they can choose to either restart the game or exit.

### Game Flow:

- The player starts at Level 1 and is asked to choose a dungeon size.
- Each level consists of navigating a dungeon grid filled with obstacles, monsters, and treasures.
- The player must avoid the monsters, reach the treasure, and complete the level.
- After each level, the grid size increases, and the difficulty rises as the player faces new challenges.
- If the player dies, they are given the option to either restart the game or quit.

### Final Note:
This game has a lot of dynamic elements such as grid shifting, random obstacles, monster movements, and treasure mechanics that make it engaging and unpredictable. I can always tweak the difficulty, rules, and interactions to make the game even more exciting!

In [195]:
main()

Streaming output truncated to the last 5000 lines.
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🧟🔲🧱🧱
🧱🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🔲🧱🔲🔲🧱🔲🔲🔲🧱
🧱🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱
🧱🔲🧱🔲🧱🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🧱
🧱🧱🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🧱
🧱🔲🔲🧱🔲🔲🧱🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱🧱🧱🔲🔲🔲🧱🧱🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🧱🔲🧱
🧱🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🔲🔲🧱🔲🧱
🧱🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🧱🧱🧱🔲🧱
🧱🔲🔲🧱🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🧱🔲🔲🔲🔲🧱🔲🧱
🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱
Moves counter: 6 and Total Moves: 192
Treasure Reappearance Counter: 3
Position in monster move:  (7, 7)
Position in monster move:  (4, 17)
Position in monster move:  (10, 21)
Monsters moved:
🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🧱🔲🔲🧱🧱🔲🧱🔲🔲🔲🧱🔲🔲🧱
🧱🔲🔲🔲🧱🔲🧱🔲🔲🔲🧱🔲🧱🔲🔲🔲🧱🔲🔲🔲🧱🧱🔲🧱🔲🧱
🧱🔲🔲🔲🧱🔲🧱🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🧱🔲🔲🔲🧱🔲🧱🔲🧱🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧟🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱
🧱👩🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🧟🔲🔲🔲🔲🔲🔲🔲🧱🧱🔲🧱🔲🧱🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🧱🔲🔲🔲🧱🔲🧱🔲🔲🔲🔲🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🧱🧱🔲🧱🧱🔲🧱🔲🔲🧱
🧱🧱🔲🔲🔲🔲🔲🔲🔲🧱🔲🧱🔲🔲🔲🔲🔲🧱🧱🔲🔲🔲🧟🔲🔲🧱
🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🧱
🧱🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🧱🔲🔲🔲🧱
🧱